In [3]:
import pandas as pd
import xgboost as xgb
import numpy as n
from sklearn.metrics import mean_squared_error
import numpy as np
from datetime import datetime, timedelta, timezone
from data_extraction.dummy_data_extractor import extract_dummy_data

def xgboost_1(df, length_of_missing_data, data_logging_interval, dqStart):
  
    # Keep only the first two columns
    df = df.iloc[:, :2]

    df.set_index(df.columns[0], inplace=True, drop=True)
    
    df = df[df.index < dqStart]

    df.reset_index(inplace=True)

    # Rename columns
    df.columns = ['ds','y']

 

    # Extract numerical features from datetime
    df['year'] = df['ds'].dt.year
    df['month'] = df['ds'].dt.month
    df['day'] = df['ds'].dt.day
    df['hour'] = df['ds'].dt.hour
    df['minute'] = df['ds'].dt.minute

    # Create future DataFrame starting from dq_start
    future_periods = int(length_of_missing_data / data_logging_interval) + 1
   

    future_temp = pd.DataFrame()
    future_temp['ds'] = [dqStart + timedelta(minutes=5 * i) for i in range(future_periods)]


    # Extract features for XGBoost
    future_temp['year'] = future_temp['ds'].dt.year
    future_temp['month'] = future_temp['ds'].dt.month
    future_temp['day'] = future_temp['ds'].dt.day
    future_temp['hour'] = future_temp['ds'].dt.hour
    future_temp['minute'] = future_temp['ds'].dt.minute

    # Initialize XGBoost model
    model_temp = xgb.XGBRegressor()

    # Fit the model
    model_temp.fit(df[['year', 'month', 'day', 'hour', 'minute']], df['y'])

    # Predict the future values
    future_temp['yhat'] = model_temp.predict(future_temp[['year', 'month', 'day', 'hour', 'minute']])

    # Filter predictions to start from dq_start
    predictions = future_temp[['ds', 'yhat']]

    # Rename columns
    predictions.columns = ['timestamp','xgboost1']

    # Set 'ds' as the index
    predictions.set_index('timestamp', inplace=True)

    return predictions



In [4]:
# Example usage:
master_table = extract_dummy_data("dummy_data")
df = master_table.at[0, "his"].iloc[:, :2]

# Extract values from the second row of master_table
length_of_missing_data = master_table.at[0, "dqDuration"]
data_logging_interval = master_table.at[0, "pointInterval"]
dqStart = master_table.at[0, "dqStart"]

# Call the function
predictions = xgboost_1(df, length_of_missing_data, data_logging_interval, dqStart)

# Display the resulting dataframe
print(predictions)

                            xgboost1
timestamp                           
2023-05-09 23:55:00+04:00  16.148209
2023-05-10 00:00:00+04:00  17.513329
2023-05-10 00:05:00+04:00  18.650255
2023-05-10 00:10:00+04:00  16.625755
2023-05-10 00:15:00+04:00  16.520821
...                              ...
2023-05-11 23:40:00+04:00  16.244705
2023-05-11 23:45:00+04:00  16.240019
2023-05-11 23:50:00+04:00  16.260220
2023-05-11 23:55:00+04:00  16.266781
2023-05-12 00:00:00+04:00  17.881887

[578 rows x 1 columns]


c:\Users\syounas\OneDrive - Enova Facilities Management\Tasks\GitHub\HubgradeDataCleaning\project\models\data_extraction\dummy_data_extractor.py:16: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2023-05-09 23:55:00+04:00' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  pythonDF.loc[i, 'dqStart'] = pd.to_datetime(df['ts'].iloc[i], format="%Y-%m-%dT%H:%M:%S%z Dubai")
